<a href="https://colab.research.google.com/github/manishbayesian/bayesianbookpub/blob/main/Book_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme(style="darkgrid")
pd.set_option('display.precision', 3)
pd.set_option('display.float_format',  '{:,.3f}'.format)

# Book Code

## Chapter 1. Bayes Calculator

In [3]:
import pandas as pd
import numpy as np

class BayesCalculator:
  def __init__(self, hypotheses, prior):
    """Initialize with hypotheses
   
    hypotheses: [X1, X2, ..., Xn]
    prior: [Pr(X1),Pr(X2),...,Pr(Xn)]
    """
    prior = np.asarray(prior)/sum(prior)
    self._bayes_df = pd.DataFrame(prior, index=hypotheses, columns=['prior'])

  def as_table(self):
    return self._bayes_df
  
  def update(self, data, likelihood):
    """Calculate evidence Pr(D) and 
       update posterior [Pr(X1|D),Pr(X2|D),...,Pr(Xn|D)]

    data: D
    likelihood:[Pr(D|X1),Pr(D|X2),...,Pr(D|Xn)]

    """
    self._bayes_df['likelihood'] = np.asarray(likelihood)
    unscaled_posterior = self._bayes_df['prior'] * likelihood    
    self._bayes_df['evidence']  = sum(unscaled_posterior)
    self._bayes_df['posterior'] = unscaled_posterior/sum(unscaled_posterior)
    return self


### Chapter 1. Coin Bias Example

In [4]:
coin_toss = BayesCalculator(['U','B'],prior=[0.5,0.5]) \
              .update('HHH',likelihood=[0.5*0.5*0.5, 1.0])
display(coin_toss.as_table())

,prior,likelihood,evidence,posterior
U,0.500,0.125,0.562,0.111
B,0.500,1.000,0.562,0.889


## Chapter 1. Covid Test Example



In [12]:
specificity, sensitivity = 0.975, 0.943
covid_test = BayesCalculator(['not infected','infected'],prior=[0.95,0.05]) \
              .update('+ve', likelihood = [1-specificity, sensitivity])
display(covid_test.as_table())

,prior,likelihood,evidence,posterior
not infected,0.950,0.025,0.071,0.335
infected,0.050,0.943,0.071,0.665


In [14]:
# For rapid antigen test, eported from https://pubmed.ncbi.nlm.nih.gov/34242764/
# Assume prevalance rate of 5%
specificity, sensitivity = 0.995, 0.653
covid_test = BayesCalculator(['not infected','infected'],prior=[0.95,0.05]) \
              .update('+ve', likelihood = [1-specificity, sensitivity])
display(covid_test.as_table())

,prior,likelihood,evidence,posterior
not infected,0.950,0.005,0.037,0.127
infected,0.050,0.653,0.037,0.873


## Chapter 1. Monty Hall Problem





In [17]:
montyhall = BayesCalculator(['Door1','Door2', 'Door3'],
                            prior=[1./3,1./3,1./3]) \
              .update('Monty Shows Door 2',
                      likelihood=[1./2, 0, 1.])
display(montyhall.as_table())

,prior,likelihood,evidence,posterior
Door1 has prize,0.333,0.500,0.500,0.333
Door2 has prize,0.333,0.000,0.500,0.000
Door3 has prize,0.333,1.000,0.500,0.667


## Chapter 1. Prosecutor's Fallacy

In [19]:
defendant = BayesCalculator(['Innocent','Guilty'],
                            prior=[9999./10000,1./10000]) \
              .update('Blood Match',
                      likelihood=[0.1, 1.])
display(defendant.as_table())

,prior,likelihood,evidence,posterior
Innocent,1.000,0.100,0.100,0.999
Guilty,0.000,1.000,0.100,0.001


## Chapter 1 - Recursive Bayes Calculator

In [7]:
class RecursiveBayesCalculator(BayesCalculator):
  def __init__(self, hypotheses, prior):
    super().__init__(hypotheses, prior)
    self._bayes_df['posterior'] = self._bayes_df['prior']

  def update(self, data, likelihood):
    self._bayes_df['prior'] = self._bayes_df['posterior']
    return super().update(data, likelihood)

In [9]:
rcoin_toss = RecursiveBayesCalculator(['U','B'],prior=[0.5,0.5]) \
              .update('H',likelihood=[0.5, 1.0]) \
              .update('H',likelihood=[0.5, 1.0]) \
              .update('H',likelihood=[0.5, 1.0])
display(rcoin_toss.as_table()[['posterior']])

,posterior
U,0.111
B,0.889


## Chapter 1 - Recursive Bayes for COVID Test Example

In [18]:
specificity, sensitivity = 0.975, 0.943
rcovid_test = RecursiveBayesCalculator(['not infected','infected'],prior=[0.95,0.05]) \
               .update('+ve',   likelihood = [1-specificity, sensitivity]) \
               .update('fever', likelihood = [0.01, 0.89])
display(rcovid_test.as_table()[['posterior']])

,posterior
not infected,0.006
infected,0.994
